In [1]:
from jupyter_cadquery import set_defaults, open_viewer

set_defaults(theme="dark")
open_viewer("CadQuery")

import math
import cadquery as cq
from cq_gears import SpurGear, RingGear
import faulthandler

faulthandler.enable()
from jupyter_cadquery.viewer.client import show

Overwriting auto display for cadquery Workplane and Shape


In [64]:
main_arm_length = 150
secondary_arm_length = main_arm_length

arm_height = 10
arm_thickness = 10
arm_extra_length = 10

gear_size = main_arm_length
gear_height = 10

gear_hole = 10
gear_hole_tolerance = 0.5
gear_spoke_width = 15

In [65]:
main_arm = (
    cq.Workplane("XY")
    .center(main_arm_length / 2 - main_arm_length / 2 - arm_extra_length / 2, 0)
    .rect(main_arm_length + arm_extra_length * 2, arm_thickness * 2)
    .extrude(arm_height)
)

main_arm_gear_hub_1 = (
    main_arm.faces(">Z")
    .workplane()
    .center(-main_arm_length / 2, 0)
    .circle(gear_hole / 2 - gear_hole_tolerance)
    .extrude(gear_height * 2)
)

main_arm_gear_hub_2 = (
    main_arm.faces(">Z")
    .workplane()
    .center(main_arm_length / 2, 0)
    .circle(gear_hole / 2 - gear_hole_tolerance)
    .extrude(gear_height * 2)
)

arm = main_arm.union(main_arm_gear_hub_1).union(main_arm_gear_hub_2)
show(arm)

. sending ... done


In [66]:
gear_arm_forward = SpurGear(
    module=1.0,
    teeth_number=gear_size,
    width=gear_height,
    helix_angle=30,
    bore_d=gear_hole,
    n_spokes=5,
    spokes_id=35,
    spokes_od=gear_size - 30,
    spoke_width=gear_spoke_width,
    # spoke_fillet=4.0,
)
gear_arm_backward = SpurGear(
    module=1.0,
    teeth_number=gear_size,
    width=gear_height,
    helix_angle=-30,
    bore_d=gear_hole,
    n_spokes=5,
    spokes_id=35,
    spokes_od=gear_size - 30,
    spoke_width=gear_spoke_width,
    # spoke_fillet=4.0,
)
gear_arm_forward_2 = SpurGear(
    module=1.0,
    teeth_number=gear_size,
    width=gear_height,
    helix_angle=30,
    n_spokes=5,
    spokes_id=35,
    spokes_od=gear_size - 30,
    spoke_width=gear_spoke_width,
    # spoke_fillet=4.0,
)

arm_gear_1 = (
    arm.faces(">Z[1]")
    .workplane()
    .center(-gear_size / 2, 0)
    # .circle(gear_arm_forward.r0)
    # .extrude(-gear_arm_forward.width)
    .gear(gear_arm_forward)
)

arm_gear_2 = (
    arm.faces(">Z[1]")
    .workplane()
    .center(gear_size / 2, 0)
    # .circle(gear_arm_backward.r0)
    # .extrude(-gear_arm_backward.width)
    .gear(gear_arm_backward)
)

magnet_arm = (
    arm_gear_2.faces(">Z")
    .workplane()
    .center(main_arm_length / 2 + 5, 0)
    .rect(secondary_arm_length - 5, arm_thickness * 2)
    .extrude(arm_height)
)

arm_gear_3 = (
    arm.faces("<Z")
    .workplane()
    .center(-gear_size / 2, 0)
    # .circle(gear_motor_main.r0)
    # .extrude(gear_motor_main.width)
    .gear(gear_arm_forward_2)
)

gears = arm.union(arm_gear_1).union(arm_gear_2).union(magnet_arm).union(arm_gear_3)
show(gears)

. sending ... done


In [90]:
main_motor = (
    cq.importers.importStep("C:\\Users\\James\\Documents\\git\\sand-table\\nema17.step")
    .rotate((0, 0, 0), (0, 1, 0), 270)
    .translate([-main_arm_length / 2 - arm_extra_length / 2, -gear_size / 2, -30])
)

secondary_motor = (
    cq.importers.importStep("C:\\Users\\James\\Documents\\git\\sand-table\\nema17.step")
    .rotate((0, 0, 0), (0, 1, 0), 270)
    .translate([-main_arm_length / 2 - arm_extra_length / 2, gear_size / 2, -30])
)

motors = main_motor.union(secondary_motor)

In [91]:
final = arm.union(gears).union(motors)

show(final, grid=[True, True, True])

. sending ... done
